The datasets contains data from an international online store. An A/B test was launched by another analyst but stopped before the due date and our task is to analyse the data and see if the A/B test was carried out correctly. Having the tehnical specifications and test results from the predecessor, show that within 14 days of signing up, users show better conversion for each stage of the funnel, at least 10% increase. 

In [17]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
from scipy import stats as st
import math as mth

In [18]:
marketing_events = pd.read_csv('/datasets/ab_project_marketing_events_us.csv')
users_upd = pd.read_csv('/datasets/final_ab_new_users_upd_us.csv')
events_upd = pd.read_csv('/datasets/final_ab_events_upd_us.csv')
participants_upd = pd.read_csv('/datasets/final_ab_participants_upd_us.csv')

In [19]:
print('Project marketing events')
display(marketing_events)
print('New users update')
display(users_upd)
print('Events update')
display(events_upd)
print('Participants update')
display(participants_upd)

Project marketing events


,name,regions,start_dt,finish_dt
0,Christmas&New Year Promo,"EU, N.America",2020-12-25,2021-01-03
1,St. Valentine's Day Giveaway,"EU, CIS, APAC, N.America",2020-02-14,2020-02-16
2,St. Patric's Day Promo,"EU, N.America",2020-03-17,2020-03-19
3,Easter Promo,"EU, CIS, APAC, N.America",2020-04-12,2020-04-19
4,4th of July Promo,N.America,2020-07-04,2020-07-11
5,Black Friday Ads Campaign,"EU, CIS, APAC, N.America",2020-11-26,2020-12-01
6,Chinese New Year Promo,APAC,2020-01-25,2020-02-07
7,Labor day (May 1st) Ads Campaign,"EU, CIS, APAC",2020-05-01,2020-05-03
8,International Women's Day Promo,"EU, CIS, APAC",2020-03-08,2020-03-10
9,Victory Day CIS (May 9th) Event,CIS,2020-05-09,2020-05-11


New users update


,user_id,first_date,region,device
0,D72A72121175D8BE,2020-12-07,EU,PC
1,F1C668619DFE6E65,2020-12-07,N.America,Android
2,2E1BF1D4C37EA01F,2020-12-07,EU,PC
3,50734A22C0C63768,2020-12-07,EU,iPhone
4,E1BDDCE0DAFA2679,2020-12-07,N.America,iPhone
...,...,...,...,...
58698,1DB53B933257165D,2020-12-20,EU,Android
58699,538643EB4527ED03,2020-12-20,EU,Mac
58700,7ADEE837D5D8CBBD,2020-12-20,EU,PC
58701,1C7D23927835213F,2020-12-20,EU,iPhone


Events update


,user_id,event_dt,event_name,details
0,E1BDDCE0DAFA2679,2020-12-07 20:22:03,purchase,99.99
1,7B6452F081F49504,2020-12-07 09:22:53,purchase,9.99
2,9CD9F34546DF254C,2020-12-07 12:59:29,purchase,4.99
3,96F27A054B191457,2020-12-07 04:02:40,purchase,4.99
4,1FD7660FDF94CA1F,2020-12-07 10:15:09,purchase,4.99
...,...,...,...,...
423756,245E85F65C358E08,2020-12-30 19:35:55,login,NaN
423757,9385A108F5A0A7A7,2020-12-30 10:54:15,login,NaN
423758,DB650B7559AC6EAC,2020-12-30 10:59:09,login,NaN
423759,F80C9BDDEA02E53C,2020-12-30 09:53:39,login,NaN


Participants update


,user_id,group,ab_test
0,D1ABA3E2887B6A73,A,recommender_system_test
1,A7A3664BD6242119,A,recommender_system_test
2,DABC14FDDFADD29E,A,recommender_system_test
3,04988C5DF189632E,A,recommender_system_test
4,4FF2998A348C484F,A,recommender_system_test
...,...,...,...
14520,1D302F8688B91781,B,interface_eu_test
14521,3DE51B726983B657,A,interface_eu_test
14522,F501F79D332BE86C,A,interface_eu_test
14523,63FBE257B05F2245,A,interface_eu_test


In [20]:
display(marketing_events.info())
display(users_upd.info())
display(events_upd.info())
display(participants_upd.info())

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 14 entries, 0 to 13
Data columns (total 4 columns):
name         14 non-null object
regions      14 non-null object
start_dt     14 non-null object
finish_dt    14 non-null object
dtypes: object(4)
memory usage: 576.0+ bytes


None

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 58703 entries, 0 to 58702
Data columns (total 4 columns):
user_id       58703 non-null object
first_date    58703 non-null object
region        58703 non-null object
device        58703 non-null object
dtypes: object(4)
memory usage: 1.8+ MB


None

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 423761 entries, 0 to 423760
Data columns (total 4 columns):
user_id       423761 non-null object
event_dt      423761 non-null object
event_name    423761 non-null object
details       60314 non-null float64
dtypes: float64(1), object(3)
memory usage: 12.9+ MB


None

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 14525 entries, 0 to 14524
Data columns (total 3 columns):
user_id    14525 non-null object
group      14525 non-null object
ab_test    14525 non-null object
dtypes: object(3)
memory usage: 340.6+ KB


None

Changing dates to the right datatype. From object to datetime.

In [21]:
marketing_events['start_dt'] = pd.to_datetime(marketing_events['start_dt'])
marketing_events['finish_dt'] = pd.to_datetime(marketing_events['finish_dt'])

users_upd ['first_date'] = pd.to_datetime(users_upd['first_date'])

events_upd ['event_dt'] = pd.to_datetime(events_upd['event_dt'])


In [22]:
display(marketing_events.info())
display(users_upd.info())
display(events_upd.info())
display(participants_upd.info())

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 14 entries, 0 to 13
Data columns (total 4 columns):
name         14 non-null object
regions      14 non-null object
start_dt     14 non-null datetime64[ns]
finish_dt    14 non-null datetime64[ns]
dtypes: datetime64[ns](2), object(2)
memory usage: 576.0+ bytes


None

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 58703 entries, 0 to 58702
Data columns (total 4 columns):
user_id       58703 non-null object
first_date    58703 non-null datetime64[ns]
region        58703 non-null object
device        58703 non-null object
dtypes: datetime64[ns](1), object(3)
memory usage: 1.8+ MB


None

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 423761 entries, 0 to 423760
Data columns (total 4 columns):
user_id       423761 non-null object
event_dt      423761 non-null datetime64[ns]
event_name    423761 non-null object
details       60314 non-null float64
dtypes: datetime64[ns](1), float64(1), object(2)
memory usage: 12.9+ MB


None

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 14525 entries, 0 to 14524
Data columns (total 3 columns):
user_id    14525 non-null object
group      14525 non-null object
ab_test    14525 non-null object
dtypes: object(3)
memory usage: 340.6+ KB


None

In [23]:
display(marketing_events.isnull().sum())
display(marketing_events.duplicated().sum())

display(users_upd.isnull().sum())
display(users_upd.duplicated().sum())

display(events_upd.isnull().sum())
display(events_upd.duplicated().sum())

display(participants_upd.isnull().sum())
display(participants_upd.duplicated().sum())

name         0
regions      0
start_dt     0
finish_dt    0
dtype: int64

0

user_id       0
first_date    0
region        0
device        0
dtype: int64

0

user_id            0
event_dt           0
event_name         0
details       363447
dtype: int64

0

user_id    0
group      0
ab_test    0
dtype: int64

0

In [24]:
events_upd = events_upd.fillna(0)
events_upd.isnull().sum()

user_id       0
event_dt      0
event_name    0
details       0
dtype: int64

There were missing values in details column and since that is a column with optional informations that are not affecting the dataset, I just filled the missing values with 0. 

#### Study conversion at different funnel stages

In [25]:
conv_by_tests = events_upd.merge(participants_upd, on='user_id')

event2 = conv_by_tests.groupby(['event_name','ab_test'])['user_id'].count()
print(event2)
print('Interface_eu_test')
conversion_page1 = event2[4] / event2[0]
print('product page','{:.1%}'.format(conversion_page1))
conversion_cart1 = event2[2] / event2[0]
print('product cart','{:.1%}'.format(conversion_cart1))
conversion_purchase1 = event2[6] / event2[0]
print('purchase ','{:.1%}'.format(conversion_purchase1))

print('Recommender_system_test')
conversion_page2 = event2[5] / event2[1]
print('product page','{:.1%}'.format(conversion_page2))
conversion_cart2 = event2[3] / event2[1]
print('product cart','{:.1%}'.format(conversion_cart2))
conversion_purchase2 = event2[7] / event2[1]
print('purchase ','{:.1%}'.format(conversion_purchase2))

event_name    ab_test                
login         interface_eu_test          33868
              recommender_system_test    10837
product_cart  interface_eu_test          11158
              recommender_system_test     3160
product_page  interface_eu_test          22259
              recommender_system_test     6702
purchase      interface_eu_test          11644
              recommender_system_test     3210
Name: user_id, dtype: int64
Interface_eu_test
product page 65.7%
product cart 32.9%
purchase  34.4%
Recommender_system_test
product page 61.8%
product cart 29.2%
purchase  29.6%


#### Is the number of events per user distributed equally in the samples?


In [26]:
merged_datas = events_upd.merge(participants_upd, on = 'user_id')
merged_data = merged_datas.merge(users_upd, on = 'user_id')

display(merged_data.groupby(['ab_test'])['event_name'].count())
merged_data.groupby(['user_id','ab_test','group'])['event_name'].count().head(15)

ab_test
interface_eu_test          78929
recommender_system_test    23909
Name: event_name, dtype: int64

user_id           ab_test                  group
0002CE61FF2C4011  interface_eu_test        A        12
001064FEAAB631A1  interface_eu_test        B         6
                  recommender_system_test  B         6
0010A1C096941592  recommender_system_test  A        12
001E72F50D1C48FA  interface_eu_test        B         6
002412F1EB3F6E38  interface_eu_test        A         6
002540BE89C930FB  interface_eu_test        A         9
0031F1B5E9FBF708  interface_eu_test        B         4
003346BB64227D0C  interface_eu_test        A         3
00341D8401F0F665  interface_eu_test        A         2
                  recommender_system_test  A         2
0036BE15EE4D319D  interface_eu_test        B         2
003DF44D7589BBD4  recommender_system_test  A        15
003F86A34B575D27  interface_eu_test        B         4
004C58ADE7CA8C4A  interface_eu_test        A         6
Name: event_name, dtype: int64

Yes, there are differences between the number of events of the users since there could be from 1 to 15 events for one user but this could be considered normal.So, the values look normal. 

#### Are there users who enter both samples?

In [27]:

both = participants_upd.pivot_table(index='user_id',values='ab_test', aggfunc='count')
print(both[both['ab_test']>1])

new =users_upd.merge(participants_upd,on='user_id')
final_merge = events_upd.merge(participants_upd, on='user_id')
newDate = final_merge[final_merge['ab_test']=='interface_eu_test']
newDate2 =final_merge[final_merge['ab_test']=='recommender_system_test']
a =new.groupby(['user_id']).agg({'group':'unique'})
b =newDate.groupby(['user_id']).agg({'group':'unique'})
c =newDate2.groupby(['user_id']).agg({'group':'unique'})

display(a['group'].value_counts())
display(b['group'].value_counts())
display(c['group'].value_counts())

                  ab_test
user_id                  
001064FEAAB631A1        2
00341D8401F0F665        2
0082295A41A867B5        2
00E68F103C66C1F7        2
00EFA157F7B6E1C4        2
...                   ...
FEA0C585A53E7027        2
FEC0BCA6C323872F        2
FF2174A1AA0EAD20        2
FF44696E39039D29        2
FFF28D02B1EACBE1        2

[887 rows x 1 columns]


[A]       7433
[B]       5764
[B, A]       1
[A, B]       1
[A, B]       1
          ... 
[B, A]       1
[B, A]       1
[A, B]       1
[A, B]       1
[A, B]       1
Name: group, Length: 443, dtype: int64

[A]    5467
[B]    5383
Name: group, dtype: int64

[A]    2747
[B]     928
Name: group, dtype: int64

Yes, some users are part of both tests but not part of both groups for the same test.

#### How is the number of events distributed by days?

In [28]:
merged_data['event_day'] = merged_data['event_dt'].dt.day
merged_data.pivot_table(index='event_day',columns='ab_test', values='event_name', aggfunc='count')

ab_test,interface_eu_test,recommender_system_test
event_day,,
7,2068,709
8,2311,593
9,2333,746
10,2551,613
11,2603,542
12,3482,558
13,3974,504
14,4783,1355
15,4333,1316


The values for both tests are increasing untill the 24 day and then are slowly decreasing up to very small values.

#### Think of the possible details in the data that you have to take into account before starting the A/B test?

In [29]:
participants_upd.groupby(['ab_test','group'])['user_id'].nunique()

ab_test                  group
interface_eu_test        A        5467
                         B        5383
recommender_system_test  A        2747
                         B         928
Name: user_id, dtype: int64

The number of users for the test should be close or equal but that is not the case in the suggested test, the 'recommender_system_test'. Also the number of users for the suggested test is smaller than 6000 .

In [30]:
new =users_upd.merge(participants_upd,on='user_id')

new2=new[new['region']=='EU']
display(new2.groupby('ab_test')['user_id'].nunique() /new.groupby('ab_test')['user_id'].nunique())


ab_test
interface_eu_test          1.000000
recommender_system_test    0.947211
Name: user_id, dtype: float64

Another criteria for the 'recommender_system_test' was the audience. It was expected to be 15% users from EU region. But as we see there are much more than 15%.  

As a note from the previous analysis we can say that the suggested test didn't match the expected requirements and even though the other test is also not having all the expected results, the number of users is closer for both groups so this won't affect our test. The difference in the number of users for the groups in the suggested test it could be the reason for the wrong results the previous analyst had. So, we are choosing the 'interface_eu_test' for our A/B test.

#### Use the z-criterion to check the statistical difference between the proportions

For this test we have the following hypotheses : 
* H0:'There is no difference between the A/B groups'
* H1:'There is a difference between the A/B groups'

In [31]:
def check_hypothesis(group1,group2, event, alpha=0.01):
    
    successes1=pivot[pivot.event_name==event][group1].iloc[0]
    successes2=pivot[pivot.event_name==event][group2].iloc[0]
    
    trials1=newDate[newDate.group==group1]['user_id'].nunique()
    trials2=newDate[newDate.group==group2]['user_id'].nunique()

    p1 = successes1/trials1
    p2 = successes2/trials2

    p_combined = (successes1 + successes2) / (trials1 + trials2)

    difference = p1 - p2 

    z_value = difference / mth.sqrt(p_combined * (1 - p_combined) * (1/trials1 + 1/trials2))

    distr = st.norm(0, 1)  

    z_value = difference / mth.sqrt(p_combined * (1 - p_combined) * (1/trials1 + 1/trials2))

    distr = st.norm(0, 1) 

    p_value = (1 - distr.cdf(abs(z_value))) * 2

    print('p-value: ', p_value)

    if (p_value < alpha):
        print("Reject H0 for",event, 'and groups',group1,group2)
    else:
        print("Fail to Reject H0 for", event,'and groups',group1,group2)   

In [32]:
pivot = newDate.pivot_table(index='event_name', values='user_id', columns='group', 
                            aggfunc=lambda x: x.nunique()).reset_index()

for i in pivot.event_name.unique():
    check_hypothesis('A','B', i, alpha=0.05)

p-value:  0.32103485047255065
Fail to Reject H0 for login and groups A B
p-value:  0.08746918913258162
Fail to Reject H0 for product_cart and groups A B
p-value:  0.2518419682737163
Fail to Reject H0 for product_page and groups A B
p-value:  0.014959088807690257
Reject H0 for purchase and groups A B


The results show that there is no difference between the groups for the funnels except for the purchase funnel.

The EDA showed interesting facts. <br>
The results from the convension date show that only 30% of the users are actually buying a product and also a bigger percentage for purchases then product_cart shows that clients buy without actually adding every time the product to the cart.<br>
The number of events for each user have big values which means that there are clients going through more steps before buying a product.<br>
Our results and the tehnical specifications for the suggested test were not the same.<br>
The audience wasn't 15% for EU region but in fact, was bigger than 90%.<br>
The number of test participants weren't even close to 6000. There were maximum 4000.
So having all this results, we couldn't choose the suggested test for the A/B test and we decided that the other test is fitting better.
In the test results it is clear that the results doesn't show an increase up to 10% for each funnel. Using the other test also changes results but still, the results are far from the expected results.